In [10]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import numpy as np

n = 2                                            # Number of player(alive)
m = 1                                  # Total number of sources that can deal damage    
l = 0
Dam = [1]  
k=2*n+m+l
const_1= 0.668
A = [1,1,const_1,1,1]

def Event(A):
    Damage = [0]*n
    counts = 10000
    qc = QuantumCircuit()
    q = QuantumRegister(2*(n+l+m)+1+n, 'q')
    c = ClassicalRegister(n, 'c')    
    qc.add_register(q)
    qc.add_register(c)
    for i in range (2*n+m+l+1):
        if i <n:                                                               
            qc.u3(A[i]*np.pi, 0, 0, q[i])
            qc.x(q[i+n])
            qc.u3(A[i+n]*np.pi, 0, 0, q[i+n])
        if n<=i<n+m:                                                            
            qc.x(q[i+n])
            qc.u3(A[i+n]*(1-Dam[i-n])*np.pi, 0, 0, q[i+n])
        if n+m<=i<n+m+l:
            qc.x(q[i+n])
            qc.u3(A[i+n]*np.pi, 0, 0, q[i+n])
        if i == n+m+l:
            qc.mct(q[2*n:2*n+l+m], q[2*n+m+l+1], q[3*n+m+l+2:3*n+2*l+2*m+1])
        if n+m+l<i<2*n+m+l+1:        
            qc.ccx(q[i-(n+m+l+1)], q[i-(m+l+1)], q[2*n+m+l])           
            qc.ccx(q[2*n+m+l], q[2*n+m+l+1], q[i+n+1])
            qc.ccx(q[i-(n+m+l+1)], q[i-(m+l+1)], q[2*n+m+l])
            qc.measure(q[i+1+n], c[2*n+m+l-i])
    qc.draw()
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend,shots = counts)
    job_result = job.result()
    A = list(job_result.get_counts(qc).keys())
    for i in A:
        B = list(i)
        for j in range (n):
            if B[j] == '1':
                Damage[j]+= int(job_result.get_counts(qc).get(i)) /counts
    return Damage

print(Event(A))

[0.254, 0]


In [73]:
import pygame
pygame.init()

xmin = ymin = 0

ymax = 700
p_y = 0.05*ymax
d_x = ymax/2-p_y#450
xmax = d_x*3
r_1 = 0.320*ymax                                        # Constant for the division between the circuit and gates
s_d = 0.112*ymax                                           # Constatnt for grids in the first 2 box 
win = pygame.display.set_mode((int(xmax),int(ymax)))
pygame.display.set_caption("QGame_1")
run = True
while run:
    pygame.time.delay(10)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    win.fill((255,255,255))
    
    # Structure
    pygame.draw.line(win, (0,0,0), (0, ymax/2),(xmax,ymax/2), 2)   
    pygame.draw.line(win, (0,0,0), (d_x,p_y),(d_x,ymax/2), 5)
    pygame.draw.line(win, (0,0,0), (d_x*2,p_y),(d_x*2,ymax/2), 5)  
    pygame.draw.line(win, (0,0,0), (d_x,ymax/2+p_y),(d_x,ymax), 5)  
    pygame.draw.line(win, (0,0,0), (d_x*2,ymax/2+p_y),(d_x*2,ymax), 5)  

    pygame.draw.line(win, (0,0,0), (0,p_y),(xmax,p_y), 2)
    pygame.draw.line(win, (0,0,0), (0,ymax/2+p_y),(xmax,ymax/2+p_y), 2)
    # Division between the last box
    pygame.draw.line(win, (0,0,0), (d_x*2,r_1+p_y),(xmax,r_1+p_y), 2)
    pygame.draw.line(win, (0,0,0), (d_x*2,r_1+p_y+ymax/2),(xmax,r_1+p_y+ymax/2), 2)
    
    
    # Section in First and second Box 
    def grid(x,y):
        for i in range(1,4):
            pygame.draw.line(win, (0,0,0), (x,s_d*i+y),(x+d_x,s_d*i+y), 2)
            pygame.draw.line(win, (0,0,0), (s_d*i+x,y),(s_d*i+x,y+d_x), 2)
        return()

    for i in range(2):
        for j in range(2):
            grid(d_x*i,j*ymax/2+p_y)
            
            
    H = pygame.image.load(r'Resource\H.jpg')
    X = pygame.image.load(r'Resource\X.jpg')
    D = pygame.image.load(r'Resource\Dot.jpg')
    edge = int(0.07*ymax)
    H = pygame.transform.scale(H, (edge, edge))
    X = pygame.transform.scale(X, (edge, edge))
    D = pygame.transform.scale(D, (edge, edge))
    
    #Gates Position
    def gates(x,y):
        win.blit(H, (x+0.03*ymax, y+0.03*ymax)) 
        win.blit(X, (x+0.13*ymax, y+0.03*ymax)) 
        win.blit(D, (x+0.23*ymax, y+0.03*ymax)) 
        return()
    gates(0.9*ymax,ymax*0.370)
    gates(0.9*ymax,ymax*0.870)
    
    # Qubit Wire
    for i in range (4):
        pygame.draw.line(win, (0,0,0), (d_x*2+0.0148148*xmax,0.09*ymax+0.08*ymax*i),((1-0.0148148)*xmax,0.09*ymax+0.08*ymax*i), 2)
        pygame.draw.line(win, (0,0,0), (d_x*2+0.0148148*xmax,0.59*ymax+0.08*ymax*i),((1-0.0148148)*xmax,0.59*ymax+0.08*ymax*i), 2)

    
    
    
    
    pygame.display.update() 
    
pygame.quit()

In [60]:
70/(1000)

0.07